# Voxel VAE-GAN Training

This notebook is designed to provide a wholistic vae-gan training experience. You can adjust the model and training parameters through the sacred configuration file, you can view training progress in tensorboard, and you can (wip) create reconstructions with the saved models!

References:

* https://github.com/anitan0925/vaegan/blob/master/examples/train.py
  * Runs 20 epochs on separate VAE and GAN then 200 on VAEGAN
* https://github.com/jlindsey15/VAEGAN/blob/master/main.py
  * Almost clear code for vaegan paper
* https://arxiv.org/pdf/1512.09300.pdf
  * vaegan paper
* https://github.com/timsainb/Tensorflow-MultiGPU-VAE-GAN
  * Best code yet!
  

## Setup

In [1]:
import env
from train_vaegan import train_vaegan
from data.thingi10k import Thingi10k
from data.modelnet10 import ModelNet10
from data import MODELNET10_TOILET_INDEX, MODELNET10_SOFA_INDEX, MODELNET10_SOFA_TOILET_INDEX
from models import MODEL_DIR


# plot things
%matplotlib inline
# autoreload modules
%load_ext autoreload
%autoreload 2

## Prepare Sacred Experiment

In [2]:
from sacred.observers import FileStorageObserver
from sacred import Experiment
import os

ex = Experiment(name='voxel_vaegan_notebook', interactive=True)
ex.observers.append(FileStorageObserver.create('experiments_vaegan'))

@ex.main
def run_experiment(cfg):
    train_vaegan(cfg)

import datetime
last_model_dir = None

## Prepare Model Config

The model dir is generated with a timestamp. This keeps you from overwriting past results and keeps results separate to avoid confusing tensorboard.

But be warned! These model dirs can take up space, so you might need to periodically go back and delete ones you do not care about.

Also, if you ever train a model that you would really like to keep, I recommend moving it to a new directory with a special name like "best_model_ever".

In [3]:
DATASET_CLASS = 'ModelNet10'
#INDEX = MODELNET10_SOFA_TOILET_INDEX
#INDEX = MODELNET10_SOFA_INDEX
INDEX = MODELNET10_TOILET_INDEX

def make_cfg():
    model_dir = os.path.join(
        MODEL_DIR,
        'voxel_vaegan1/modelnet10/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')))
    print(model_dir)
    os.makedirs(model_dir)

    cfg = {
        'cfg': {
            "dataset": {
                "class": DATASET_CLASS,
                "index": INDEX,
                #"tag": "animal",
                #"filter_id": 126660,
                #"pctile": 1.0,
                "splits": True
                #"splits": {
                #    "train": .8,
                #    "dev": .1,
                #    "test": .1
                #}
            },
            "generator": {
                "verbose": True,
                "pad": True
            }, 
            "model": {
                "ckpt_dir": model_dir,
                "voxels_dim": 32,
                "batch_size": 32,
                # Do 0.0001 for 1 epoch, then 0.001 for rest of training
                #"learning_rate": [(1, 0.0001), (None, 0.001)],
                #"learning_rate": 0.0001,
                "enc_lr": 0.0001,
                "dec_lr": 0.0001,
                "dis_lr": 0.0001,
                "epochs": 201,
                "keep_prob": 1.0,
                "kl_div_loss_weight": 100,
                "recon_loss_weight": 10000,
                "ll_weight": .0001,
                "dec_weight": 100,
                "latent_dim": 100,
                "verbose": True,
                "debug": False,
                "input_repeats": 1,
                "display_step": 1,
                #"example_stl_id": 126660,
                "voxel_prob_threshold": 0.065,
                "dev_step": 10,
                "save_step": 10,
                'launch_tensorboard': True,
                'tb_dir': 'tb',
                #'tb_compare': [('best_sofa_and_toilet', '/home/jcworkma/jack/3d-form/models/voxel_vaegan1/modelnet10/2019-03-15_17-08-43/tb')],
                #'tb_compare': [('best_vaegan', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-17_08-40-29/tb')],
                'tb_compare': [('vaegan_100epochs_toilets', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-18_13-12-53/tb')],
                'no_gan': False,
                'monitor_memory': True,
                # these settings control how often the components' optimizers are executed during the training loop
                'train_vae_cadence': 1,
                'train_gan_cadence': 1
            }
        }
    }
    
    return cfg

## Tensorboard Prep

We launch tensorboard with a call to the python subprocess module. Sometimes, that process does not die with the rest of the experiment and lingers on as a system process. This becomes a problem when we try to initialize tensorboard for the next experiment because they cannot share the same port!

The function below is designed to solve this problem. It uses the linux pgrep utility to search for existing tensorboard processes and kill them. Note that this probably won't work on Windows.

In [4]:
from utils import kill_tensorboard

kill_tensorboard()

['pgrep', 'tensorboard'] yielded -> b''


## Training

We start with a check that we are not attempting to overwrite the last MODEL_DIR. If you are blocked by the assert, re-execute the cfg code above to generate a new MODEL_DIR. This will allow you to move ahead with training.

The sacred experiment will save away a copy of your experiment settings in an experiments directory. This can be accessed later in case we need to retrieve a prime config.

If tensorboard is enabled, tune in at localhost:6006 or your_ip:6006
   

In [ ]:
cfg = make_cfg()
model_dir = cfg.get('cfg').get('model').get('ckpt_dir')
kill_tensorboard()

/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-20_08-46-30
['pgrep', 'tensorboard'] yielded -> b''


In [ ]:
if last_model_dir == model_dir:
    print('dont overwrite!')
    assert False
else:
    last_model_dir = model_dir

ex.run(config_updates=cfg)

WARNING - root - Added new config entry: "cfg.dataset.class"
WARNING - root - Added new config entry: "cfg.dataset.index"
WARNING - root - Added new config entry: "cfg.dataset.splits"
WARNING - root - Added new config entry: "cfg.generator.pad"
WARNING - root - Added new config entry: "cfg.generator.verbose"
WARNING - root - Added new config entry: "cfg.model.batch_size"
WARNING - root - Added new config entry: "cfg.model.ckpt_dir"
WARNING - root - Added new config entry: "cfg.model.debug"
WARNING - root - Added new config entry: "cfg.model.dec_lr"
WARNING - root - Added new config entry: "cfg.model.dec_weight"
WARNING - root - Added new config entry: "cfg.model.dev_step"
WARNING - root - Added new config entry: "cfg.model.dis_lr"
WARNING - root - Added new config entry: "cfg.model.display_step"
WARNING - root - Added new config entry: "cfg.model.enc_lr"
WARNING - root - Added new config entry: "cfg.model.epochs"
WARNING - root - Added new config entry: "cfg.model.input_repeats"
WARNIN

Logging to /home/jcworkma/jack/3d-form/src/logs/2019-03-20_08-46__root.log
Starting train_vaegan main
Numpy random seed: 909862859
Saved cfg: /home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-20_08-46-30/cfg.json
Dataset: <class 'data.modelnet10.ModelNet10'>
Using dataset index /home/jcworkma/jack/3d-form/src/../data/processed/modelnet10_toilet_index.csv and pctile None
Shuffling dataset
dataset n_input=7104
Splitting Datasets
Num input = 7104
Num batches per epoch = 222.00
Initializing VoxelVaegan
['tensorboard', '--logdir', 'current:/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-20_08-46-30/tb,vaegan_100epochs_toilets:/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-18_13-12-53/tb']
Epoch: 0, Elapsed Time: 0.03
Training VAE in this epoch
Training GAN in this epoch
Epoch: 0 / 201, Batch: 0 (0 / 32), Elapsed time: 0.03 mins
Enc Loss = 17.80, KL Divergence = 0.18, Reconstruction Loss = 0.11, ll_loss = 0.01, d

Enc Loss = 128.82, KL Divergence = 0.01, Reconstruction Loss = 0.12, ll_loss = 1275824.62, dis_Loss = 0.00, dec_Loss = 127.58, Elapsed time: 2.88 mins
Memory Use (GB): 1.5007171630859375
Epoch: 0 / 201, Batch: 31 (0 / 1024), Elapsed time: 2.88 mins
Enc Loss = 130.96, KL Divergence = 0.01, Reconstruction Loss = 0.12, ll_loss = 1298223.00, dis_Loss = 0.00, dec_Loss = 129.83, Elapsed time: 2.97 mins
Memory Use (GB): 1.7259101867675781
Epoch: 0 / 201, Batch: 32 (0 / 1056), Elapsed time: 2.97 mins
Enc Loss = 132.16, KL Divergence = 0.01, Reconstruction Loss = 0.12, ll_loss = 1309675.25, dis_Loss = 0.00, dec_Loss = 130.97, Elapsed time: 3.06 mins
Memory Use (GB): 1.5680770874023438
Epoch: 0 / 201, Batch: 33 (0 / 1088), Elapsed time: 3.06 mins
Enc Loss = 126.35, KL Divergence = 0.01, Reconstruction Loss = 0.12, ll_loss = 1251515.00, dis_Loss = 0.00, dec_Loss = 125.15, Elapsed time: 3.15 mins
Memory Use (GB): 1.5238761901855469
Epoch: 0 / 201, Batch: 34 (0 / 1120), Elapsed time: 3.16 mins
Enc 

Memory Use (GB): 1.563385009765625
Epoch: 0 / 201, Batch: 64 (0 / 2080), Elapsed time: 5.90 mins
Enc Loss = 86.29, KL Divergence = 0.03, Reconstruction Loss = 0.27, ll_loss = 837066.88, dis_Loss = 0.00, dec_Loss = 83.71, Elapsed time: 5.99 mins
Memory Use (GB): 1.5465660095214844
Epoch: 0 / 201, Batch: 65 (0 / 2112), Elapsed time: 5.99 mins
Enc Loss = 58.88, KL Divergence = 0.02, Reconstruction Loss = 0.25, ll_loss = 567890.50, dis_Loss = 0.00, dec_Loss = 56.79, Elapsed time: 6.08 mins
Memory Use (GB): 1.5579261779785156
Epoch: 0 / 201, Batch: 66 (0 / 2144), Elapsed time: 6.08 mins
Enc Loss = 58.57, KL Divergence = 0.01, Reconstruction Loss = 0.21, ll_loss = 570735.00, dis_Loss = 0.00, dec_Loss = 57.07, Elapsed time: 6.17 mins
Memory Use (GB): 1.3642234802246094
Epoch: 0 / 201, Batch: 67 (0 / 2176), Elapsed time: 6.17 mins
Enc Loss = 40.98, KL Divergence = 0.01, Reconstruction Loss = 0.26, ll_loss = 394889.31, dis_Loss = 0.00, dec_Loss = 39.49, Elapsed time: 6.26 mins
Memory Use (GB): 

Enc Loss = 79.79, KL Divergence = 0.01, Reconstruction Loss = 0.26, ll_loss = 782959.31, dis_Loss = 0.00, dec_Loss = 78.30, Elapsed time: 9.02 mins
Memory Use (GB): 1.5766448974609375
Epoch: 0 / 201, Batch: 98 (0 / 3168), Elapsed time: 9.02 mins
Enc Loss = 73.39, KL Divergence = 0.02, Reconstruction Loss = 0.22, ll_loss = 717642.69, dis_Loss = 0.00, dec_Loss = 71.77, Elapsed time: 9.11 mins
Memory Use (GB): 1.5455589294433594
Epoch: 0 / 201, Batch: 99 (0 / 3200), Elapsed time: 9.11 mins
Enc Loss = 117.94, KL Divergence = 0.02, Reconstruction Loss = 0.28, ll_loss = 1156268.88, dis_Loss = 0.01, dec_Loss = 115.64, Elapsed time: 9.20 mins
Memory Use (GB): 1.5962867736816406
Epoch: 0 / 201, Batch: 100 (0 / 3232), Elapsed time: 9.20 mins
Enc Loss = 67.89, KL Divergence = 0.02, Reconstruction Loss = 0.24, ll_loss = 662843.38, dis_Loss = 0.00, dec_Loss = 66.28, Elapsed time: 9.29 mins
Memory Use (GB): 1.6298027038574219
Epoch: 0 / 201, Batch: 101 (0 / 3264), Elapsed time: 9.29 mins
Enc Loss = 

...

Memory Use (GB): 1.4740524291992188
Epoch: 2 / 201, Batch: 45 (1472 / 5504), Elapsed time: 35.74 mins
Enc Loss = 353.46, KL Divergence = 0.94, Reconstruction Loss = 0.31, ll_loss = 2596067.75, dis_Loss = 0.00, dec_Loss = 259.61, Elapsed time: 35.83 mins
Memory Use (GB): 1.6135101318359375
Epoch: 2 / 201, Batch: 46 (1504 / 5504), Elapsed time: 35.83 mins
Enc Loss = 461.56, KL Divergence = 1.53, Reconstruction Loss = 0.27, ll_loss = 3081953.75, dis_Loss = 0.00, dec_Loss = 308.20, Elapsed time: 35.92 mins
Memory Use (GB): 1.5407142639160156
Epoch: 2 / 201, Batch: 47 (1536 / 5504), Elapsed time: 35.92 mins
Enc Loss = 224.52, KL Divergence = 0.55, Reconstruction Loss = 0.28, ll_loss = 1692626.50, dis_Loss = 0.00, dec_Loss = 169.26, Elapsed time: 36.01 mins
Memory Use (GB): 1.5373001098632812
Epoch: 2 / 201, Batch: 48 (1568 / 5504), Elapsed time: 36.01 mins
Enc Loss = 306.34, KL Divergence = 0.82, Reconstruction Loss = 0.26, ll_loss = 2239644.50, dis_Loss = 0.00, dec_Loss = 223.97, Elapsed t

Enc Loss = 278.76, KL Divergence = 0.82, Reconstruction Loss = 0.33, ll_loss = 1966509.00, dis_Loss = 0.00, dec_Loss = 196.65, Elapsed time: 38.78 mins
Memory Use (GB): 1.6878204345703125
Epoch: 2 / 201, Batch: 78 (2528 / 5504), Elapsed time: 38.78 mins
Enc Loss = 300.80, KL Divergence = 0.65, Reconstruction Loss = 0.27, ll_loss = 2355861.75, dis_Loss = 0.00, dec_Loss = 235.59, Elapsed time: 38.87 mins
Memory Use (GB): 1.6959152221679688
Epoch: 2 / 201, Batch: 79 (2560 / 5504), Elapsed time: 38.87 mins
Enc Loss = 218.54, KL Divergence = 0.70, Reconstruction Loss = 0.25, ll_loss = 1484849.50, dis_Loss = 0.00, dec_Loss = 148.48, Elapsed time: 38.96 mins
Memory Use (GB): 1.758087158203125
Epoch: 2 / 201, Batch: 80 (2592 / 5504), Elapsed time: 38.96 mins
Enc Loss = 318.79, KL Divergence = 1.23, Reconstruction Loss = 0.31, ll_loss = 1959573.75, dis_Loss = 0.00, dec_Loss = 195.96, Elapsed time: 39.05 mins
Memory Use (GB): 1.6441917419433594
Epoch: 2 / 201, Batch: 81 (2624 / 5504), Elapsed ti

Memory Use (GB): 1.4393959045410156
Epoch: 2 / 201, Batch: 110 (3552 / 5504), Elapsed time: 41.72 mins
Enc Loss = 308.59, KL Divergence = 0.79, Reconstruction Loss = 0.32, ll_loss = 2297924.50, dis_Loss = 0.00, dec_Loss = 229.79, Elapsed time: 41.81 mins
Memory Use (GB): 1.6515960693359375
Epoch: 2 / 201, Batch: 111 (3584 / 5504), Elapsed time: 41.81 mins
Enc Loss = 305.97, KL Divergence = 0.72, Reconstruction Loss = 0.23, ll_loss = 2337196.50, dis_Loss = 0.00, dec_Loss = 233.72, Elapsed time: 41.90 mins
Memory Use (GB): 1.5908088684082031
Epoch: 2 / 201, Batch: 112 (3616 / 5504), Elapsed time: 41.90 mins
Enc Loss = 374.64, KL Divergence = 1.08, Reconstruction Loss = 0.32, ll_loss = 2671155.00, dis_Loss = 0.00, dec_Loss = 267.12, Elapsed time: 41.99 mins
Memory Use (GB): 1.7051277160644531
Epoch: 2 / 201, Batch: 113 (3648 / 5504), Elapsed time: 42.00 mins
Enc Loss = 187.81, KL Divergence = 0.90, Reconstruction Loss = 0.27, ll_loss = 981178.75, dis_Loss = 0.00, dec_Loss = 98.12, Elapsed

Enc Loss = 220.36, KL Divergence = 0.69, Reconstruction Loss = 0.25, ll_loss = 1516219.62, dis_Loss = 0.00, dec_Loss = 151.62, Elapsed time: 44.75 mins
Memory Use (GB): 1.5933189392089844
Epoch: 2 / 201, Batch: 143 (4608 / 5504), Elapsed time: 44.75 mins
Enc Loss = 246.73, KL Divergence = 0.84, Reconstruction Loss = 0.31, ll_loss = 1624441.12, dis_Loss = 0.00, dec_Loss = 162.44, Elapsed time: 44.84 mins
Memory Use (GB): 1.641357421875
Epoch: 2 / 201, Batch: 144 (4640 / 5504), Elapsed time: 44.84 mins
Enc Loss = 240.32, KL Divergence = 1.24, Reconstruction Loss = 0.31, ll_loss = 1158630.38, dis_Loss = 0.00, dec_Loss = 115.86, Elapsed time: 44.94 mins
Memory Use (GB): 1.6175537109375
Epoch: 2 / 201, Batch: 145 (4672 / 5504), Elapsed time: 44.94 mins
Enc Loss = 399.78, KL Divergence = 1.91, Reconstruction Loss = 0.27, ll_loss = 2089501.25, dis_Loss = 0.00, dec_Loss = 208.95, Elapsed time: 45.03 mins
Memory Use (GB): 1.6836585998535156
Epoch: 2 / 201, Batch: 146 (4704 / 5504), Elapsed time

Memory Use (GB): 1.7005996704101562
Epoch: 3 / 201, Batch: 2 (96 / 5504), Elapsed time: 47.62 mins
Enc Loss = 242.94, KL Divergence = 0.97, Reconstruction Loss = 0.28, ll_loss = 1460502.75, dis_Loss = 0.00, dec_Loss = 146.05, Elapsed time: 47.71 mins
Memory Use (GB): 1.4611778259277344
Epoch: 3 / 201, Batch: 3 (128 / 5504), Elapsed time: 47.71 mins
Enc Loss = 291.45, KL Divergence = 0.87, Reconstruction Loss = 0.34, ll_loss = 2045082.25, dis_Loss = 0.00, dec_Loss = 204.51, Elapsed time: 47.81 mins
Memory Use (GB): 1.6211776733398438
Epoch: 3 / 201, Batch: 4 (160 / 5504), Elapsed time: 47.81 mins
Enc Loss = 247.12, KL Divergence = 1.12, Reconstruction Loss = 0.28, ll_loss = 1353093.50, dis_Loss = 0.00, dec_Loss = 135.31, Elapsed time: 47.90 mins
Memory Use (GB): 1.527557373046875
Epoch: 3 / 201, Batch: 5 (192 / 5504), Elapsed time: 47.90 mins
Enc Loss = 304.05, KL Divergence = 0.97, Reconstruction Loss = 0.28, ll_loss = 2072223.75, dis_Loss = 0.00, dec_Loss = 207.22, Elapsed time: 48.00